In [1]:
%pylab inline
import nltk
import json
import re
import time
import sqlalchemy
import pandas as pd
import warnings
import isbnlib
from bs4 import BeautifulSoup
import requests
from html.parser import HTMLParser
import urllib
from urllib import urlopen

warnings.filterwarnings('ignore')

# record linkage package
import recordlinkage as rl
from recordlinkage.preprocessing import clean, phonenumbers, phonetic

from __future__ import print_function
from six.moves import zip, range 
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, roc_auc_score, auc
from sklearn import preprocessing
from collections import Counter, OrderedDict
from nltk.corpus import stopwords
from nltk import SnowballStemmer

nltk.download('stopwords')

Populating the interactive namespace from numpy and matplotlib
[nltk_data] Downloading package stopwords to
[nltk_data]     /nfshome/ag4215/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
url = 'https://thewirecutter.com/'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
data = soup.findAll(text=True)

def visible(element):
    if element.parent.name in ['style', 'script', '[document]', 'head', 'title']:
        return False
    elif re.match('<!--.*-->', str(element.encode('utf-8'))):
        return False
    return True

result = filter(visible, data)

In [3]:
RE_PREPROCESS = re.compile( r'\W+|\d+' )

In [4]:
processed_corpus = np.array( [ re.sub( RE_PREPROCESS, ' ', description ).lower() for description in result ] )

In [5]:
eng_stopwords = stopwords.words('english')

In [6]:
def create_bag_of_words( corpus,
                         NGRAM_RANGE = ( 0, 1 ),
                         stop_words = None,
                         stem = False,
                         MIN_DF = 0.005,
                         MAX_DF = 0.95,
                         USE_IDF = False ):

    """
    Turn a corpus of text into a bag-of-words.
    
    Parameters
    -----------
    corpus: ls
        test of documents in corpus    
    NGRAM_RANGE: tuple
        range of N-gram. Default (0,1)
    stop_words: ls
        list of commonly occuring words that have little semantic
        value
    stem: bool
        use a stemmer to stem words
    MIN_DF: float
       exclude words that have a frequency less than the threshold
    MAX_DF: float
        exclude words that have a frequency greater than the threshold
    
    
    Returns
    -------
    bag_of_words: scipy sparse matrix
        scipy sparse matrix of text
    features:
        ls of words
    """
    #parameters for vectorizer 
    ANALYZER = "word" #unit of features are single words rather then phrases of words 
    STRIP_ACCENTS = 'unicode'
    stemmer = nltk.SnowballStemmer("english")

    if stem:
        tokenize = lambda x: [stemmer.stem(i) for i in x.split()]
    else:
        tokenize = None
    vectorizer = CountVectorizer(analyzer=ANALYZER,
                                tokenizer=tokenize, 
                                ngram_range=NGRAM_RANGE,
                                stop_words = stop_words,
                                strip_accents=STRIP_ACCENTS,
                                min_df = MIN_DF,
                                max_df = MAX_DF)
    
    bag_of_words = vectorizer.fit_transform( corpus ) #transform our corpus is a bag of words 
    features = vectorizer.get_feature_names()

    if USE_IDF:
        NORM = None #turn on normalization flag
        SMOOTH_IDF = True #prvents division by zero errors
        SUBLINEAR_IDF = True #replace TF with 1 + log(TF)
        transformer = TfidfTransformer(norm = NORM,smooth_idf = SMOOTH_IDF,sublinear_tf = True)
        #get the bag-of-words from the vectorizer and
        #then use TFIDF to limit the tokens found throughout the text 
        tfidf = transformer.fit_transform(bag_of_words)
        
        return tfidf, features
    else:
        return bag_of_words, features

In [7]:
#create a bag of words with PROCESSED corpus
bag_of_words, features = create_bag_of_words(processed_corpus)

In [8]:
features

[u'accessories',
 u'after',
 u'all',
 u'and',
 u'appliances',
 u'baby',
 u'best',
 u'by',
 u'cars',
 u'for',
 u'gear',
 u'home',
 u'in',
 u'is',
 u'may',
 u'more',
 u'of',
 u'office',
 u'or',
 u'see',
 u'testing',
 u'that',
 u'the',
 u'to',
 u'travel',
 u'updated',
 u'we']

In [9]:
def get_word_counts( bag_of_words, feature_names ):

    """
    Get the ordered word counts from a bag_of_words
    
    Parameters
    ----------
    bag_of_words: obj
        scipy sparse matrix from CounterVectorizer
    feature_names: ls
        list of words
        
    Returns
    -------
    word_counts: dict
        Dictionary of word counts
    """

    # convert bag of words to array
    np_bag_of_words = bag_of_words.toarray()
    
    # calculate word count.
    word_count = np.sum(np_bag_of_words,axis=0)
    
    # convert to flattened array.
    np_word_count = np.asarray(word_count).ravel()
    
    # create dict of words mapped to count of occurrences of each word.
    dict_word_counts = dict( zip(feature_names, np_word_count) )
    
    # Create ordered dictionary
    orddict_word_counts = OrderedDict( sorted(dict_word_counts.items(), key=lambda x: x[1], reverse=True), )
    
    return orddict_word_counts

In [10]:
get_word_counts(bag_of_words, features)

OrderedDict([(u'the', 127),
             (u'best', 73),
             (u'and', 62),
             (u'all', 55),
             (u'we', 42),
             (u'for', 41),
             (u'of', 30),
             (u'updated', 29),
             (u'to', 28),
             (u'home', 27),
             (u'by', 26),
             (u'may', 24),
             (u'appliances', 23),
             (u'is', 22),
             (u'after', 21),
             (u'in', 20),
             (u'more', 18),
             (u'that', 18),
             (u'office', 17),
             (u'accessories', 16),
             (u'gear', 16),
             (u'baby', 16),
             (u'cars', 15),
             (u'testing', 14),
             (u'see', 14),
             (u'travel', 14),
             (u'or', 14)])

In [11]:
processed_bag_of_words, processed_features = create_bag_of_words(processed_corpus, stop_words = eng_stopwords)
dict_processed_word_counts = get_word_counts(processed_bag_of_words, processed_features)
dict_processed_word_counts

OrderedDict([(u'best', 73),
             (u'updated', 29),
             (u'home', 27),
             (u'may', 24),
             (u'appliances', 23),
             (u'office', 17),
             (u'gear', 16),
             (u'accessories', 16),
             (u'baby', 16),
             (u'cars', 15),
             (u'travel', 14),
             (u'testing', 14),
             (u'see', 14)])

In [12]:
df = pd.DataFrame.from_dict(dict_processed_word_counts, orient='index')

In [13]:
df.index.rename('Words', inplace=True)
df.columns = ['Frequency']

In [14]:
df

,Frequency
Words,
best,73
updated,29
home,27
may,24
appliances,23
office,17
gear,16
accessories,16
baby,16
